In [ ]:
import os
import glob
import numpy as np
from PIL import Image

In [ ]:
train_irrg_dir = "/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary"
train_irrg_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary"

test_irrg_dir = "/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary"
test_irrg_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary"

In [ ]:
train_irrg_list = glob.glob(train_irrg_dir+"/*.tif")
train_irrg_list = sorted(train_irrg_list)
train_irrg_list[-5:]

In [ ]:
train_irrg_label_list = glob.glob(train_irrg_label_dir+"/*.tif")
train_irrg_label_list = sorted(train_irrg_label_list)
train_irrg_label_list[-5:]

In [ ]:
test_irrg_list = glob.glob(test_irrg_dir+"/*.tif")
test_irrg_list = sorted(test_irrg_list)
test_irrg_list[-5:]

In [ ]:
test_irrg_label_list = glob.glob(test_irrg_label_dir+"/*.tif")
test_irrg_label_list = sorted(test_irrg_label_list)
test_irrg_label_list[-5:]

In [ ]:
patch_size = 512
overlap_rate = 1/3
overlap = int(512*(1/3))
print(f"patch_size: {patch_size}, overlap_rate: {overlap_rate:.5f}, overlap: {overlap}")

In [ ]:
save_train_image_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_for_participants_no_Boundary"
save_train_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_label_for_participants_no_Boundary"

save_test_image_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_for_test_no_Boundary"
save_test_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_label_for_test_no_Boundary"

# Extract Patch from Train Datset (Overlap)

In [ ]:
def extract_train_patches(image_list, label_list, save_dest_for_image, save_dest_for_label, patch_size, overlap_rate):
    
    overlap = int(patch_size*(1-overlap_rate)) # Most Important: overlap based on <patch_size>. 

    for idx, (img_path, label_path) in enumerate(zip(image_list, label_list)):
        img = Image.open(img_path)
        label = Image.open(label_path)
        assert img.size == label.size, f" Different Shape Image and Label. "
        (height, width) = img.size
        img_dir = os.path.basename(img_path).split('.')[0]
        label_dir = os.path.basename(label_path).split('.')[0]

        # extract patch start
        h = 0
        height_count=0
        extract_finish = False

        while not extract_finish:

            if h+patch_size <= height:

                # set row flag
                extract_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting row
                while extract_row:
                    if w+patch_size <= width:
                        print(f"h:{h} w:{w} ->  h':{h+patch_size} w':{w+patch_size}")
                        patch_img = img.crop([h, w, h+patch_size, w+patch_size])
                        patch_label = label.crop([h, w, h+patch_size, w+patch_size])
                    else:
                        print(f"h:{h} w:{width-patch_size} ->  h':{h+patch_size} w':{width}")                        
                        patch_img = img.crop([h, width-patch_size, h+patch_size, width])
                        patch_label = label.crop([h, width-patch_size, h+patch_size, width])
                        # stop
                        extract_row = False

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += overlap
                    width_count += 1
                    #
                    print(height_count, width_count)

            else:
                # set last row flag
                extract_last_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting Last row
                while extract_last_row:

                    if w+patch_size <= width:
                        print(f"h:{height-patch_size} w:{w} ->  h':{height} w':{w+patch_size}")
                        patch_img = img.crop([height-patch_size, w, height, w+patch_size])
                        patch_label = label.crop([height-patch_size, w, height, w+patch_size])
                    else:
                        print(f"h:{height-patch_size} w:{w-patch_size} ->  h':{height} w':{width}")
                        patch_img = img.crop([height-patch_size, width-patch_size, height, width])
                        patch_label = label.crop([height-patch_size, width-patch_size, height, width])
                        # stop
                        extract_last_row = False
                        extract_finish = True

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))
                    
                    # iter along w
                    w += overlap
                    width_count += 1
                    #
                    print(height_count, width_count)
                    
            # iter along h
            h += overlap
            height_count += 1

        # extract patch end
        print(f"Image {img_dir} extracted finished !")

In [ ]:
extract_train_patches(
    train_irrg_list, 
    train_irrg_label_list, 
    save_dest_for_image=save_train_image_dir, 
    save_dest_for_label=save_train_label_dir, 
    patch_size=512, 
    overlap_rate=1/3)

# Extract Patch from Test Dataset (No Overlap)

In [ ]:
# No Overlap
def extract_test_patches(image_list, label_list, save_dest_for_image, save_dest_for_label, patch_size,):
    
    for idx, (img_path, label_path) in enumerate(zip(image_list, label_list)):
        img = Image.open(img_path)
        label = Image.open(label_path)
        assert img.size == label.size, f" Different Shape Image and Label. "
        (height, width) = img.size
        img_dir = os.path.basename(img_path).split('.')[0]
        label_dir = os.path.basename(label_path).split('.')[0]

        # extract patch start
        h = 0
        height_count=0
        extract_finish = False

        while not extract_finish:

            if h+patch_size > height:
                # set last row flag
                extract_last_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting Last row
                while extract_last_row:
            
                    if w + patch_size > width:
                        print(f"h:{height-patch_size} w:{width-patch_size} ->  h':{height} w':{width}")
                        patch_img = img.crop([height-patch_size, width-patch_size, height, width])
                        patch_label = label.crop([height-patch_size, width-patch_size, height, width])
                        # stop
                        extract_last_row = False
                        extract_finish = True
                        
                    else:
                        print(f"h:{height-patch_size} w:{w} ->  h':{height} w':{w+patch_size}")
                        patch_img = img.crop([height-patch_size, w, height, w+patch_size])
                        patch_label = label.crop([height-patch_size, w, height, w+patch_size])

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += patch_size
                    width_count += 1
                    print(height_count, width_count)
  
            else:
                # set row flag
                extract_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting row
                while extract_row:
                    if w+patch_size > width:
                        print(f"h:{h} w:{width-patch_size} ->  h':{h+patch_size} w':{width}")
                        patch_img = img.crop([h, width-patch_size, h+patch_size, width])
                        patch_label = label.crop([h, width-patch_size, h+patch_size, width])
                        # stop
                        extract_row = False
                    else:
                        print(f"h:{h} w:{w} ->  h':{h+patch_size} w':{w+patch_size}")
                        patch_img = img.crop([h, w, h+patch_size, w+patch_size])
                        patch_label = label.crop([h, w, h+patch_size, w+patch_size])

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += patch_size
                    width_count += 1
                    print(height_count, width_count)

            # iter along h
            h += patch_size
            height_count += 1
            
        # extract patch end
        print(f"Image {img_dir} extracted finished !")

In [ ]:
extract_test_patches(
    test_irrg_list, 
    test_irrg_label_list, 
    save_dest_for_image=save_test_image_dir, 
    save_dest_for_label=save_test_label_dir, 
    patch_size=512)

In [ ]:
x=0
size = 512
num = 0

while x < 6000:
    num += 1
        
    t = x + size
    
    y = t-170
    print(f"Cor: {y} = {t}-170  Crop: [{x}, {t}]")
    
    x = y
    

print(f"extract num:{num}")